In [74]:
#Dowloading the data
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-04-01 09:29:58--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.006s  

2023-04-01 09:29:58 (185 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [75]:
with open('input.txt','r',encoding='utf-8') as f:
  text= f.read()

In [76]:
print(f"The length of characters: {len(text)}")

The length of characters: 1115394


In [77]:
#printing the fisrt 500 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [78]:
#characters which are in the whole text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [79]:
#create a mapping 
#string to interger
stoi = { ch:i for i,ch in enumerate(chars) }
#intenger to string
itos = { i:ch for i,ch in enumerate(chars) }
#take input as string, input intergers
encode = lambda s: [stoi[c] for c in s]
#take a list of interger, input string
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("Hii there"))
print(decode(encode("Hii there")))


[20, 47, 47, 1, 58, 46, 43, 56, 43]
Hii there


In [80]:
#encoding the whole dataset and storing it on a torch.tensor
import torch
data = torch.tensor(encode(text),dtype=torch.long)
print(data.shape,data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [81]:
#Spliting the data into train and validation data
n = int(0.9*len(data)) #90% of the data will be used for training
train_data = data[:n] #10% of the data will be used for validation
valid_data = data[n:]

In [82]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [83]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"When the input is {context} the target is {target}")


When the input is tensor([18]) the target is 47
When the input is tensor([18, 47]) the target is 56
When the input is tensor([18, 47, 56]) the target is 57
When the input is tensor([18, 47, 56, 57]) the target is 58
When the input is tensor([18, 47, 56, 57, 58]) the target is 1
When the input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
When the input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
When the input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [84]:
torch.manual_seed(1337)
batch_size = 4 # how indipendent sequences will process in parallel
bloack_size = 8 #what is the maximum length of predictions

def get_batch(split):
  #develop a batch of data
  data = train_data if split== 'train' else valid_data
  ix = torch.randint(len(data)-block_size,(batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y= torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x,y
  
xb,yb= get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print("=====================================================================================")

for b in range(batch_size): #batch dimension
  for t in range(block_size): #time dimension
    context = xb[b,:t+1]
    target = yb[b,t]
    print(f"when the input is {context.tolist()} the target {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
when the input is [24] the target 43
when the input is [24, 43] the target 58
when the input is [24, 43, 58] the target 5
when the input is [24, 43, 58, 5] the target 57
when the input is [24, 43, 58, 5, 57] the target 1
when the input is [24, 43, 58, 5, 57, 1] the target 46
when the input is [24, 43, 58, 5, 57, 1, 46] the target 43
when the input is [24, 43, 58, 5, 57, 1, 46, 43] the target 39
when the input is [44] the target 53
when the input is [44, 53] the target 56
when the input is [44, 53, 56] the target 1
when the input is [44, 53, 56, 1] the target 58
when the input is [44, 53, 56, 1, 58] t

In [85]:
print(xb) # the input into the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [86]:
from torch.nn.modules import transformer
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BiagramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    #each token directly reads off the logistics for the next token from the lookup table
    self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)

  def forward(self,idx,targets = None):
    #idx and intergers are both (B,T) tensor of intergers
    logits = self.token_embedding_table(idx) #(B,T,C) 4,8,65
    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits,targets)
    return logits,loss

  def generate(self,idx,max_new_tokens):
    #idx is the (B,T) is the array of indices in the current context
    for _ in range(max_new_tokens):
      #get predictions
      logits, loss = self(idx)
      #focus on the last time step
      logits = logits[:, -1, :] #becomes (B,C)
      #apply softmax to get the probabilities
      probs = F.softmax(logits,dim =-1)
      #sample the distribution
      idx_next = torch.multinomial(probs,num_samples=1) #B,C
      #append a sampled index to the running index
      idx = torch.cat((idx,idx_next),dim=1) # B,T+1
    return idx

m = BiagramLanguageModel(vocab_size)
logits,loss = m(xb,yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx=torch.zeros((1, 1),dtype= torch.long),max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [96]:
#create a PyTorch Optimizer
optimizer = torch.optim.AdamW(m.parameters(),lr=1e-5)

In [97]:
batch_size = 32
for steps in range(100000): #increase number of steps for good results
  #sample a batch data
    xb,yb = get_batch('train')

  #evaluate the loss
    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward
    optimizer.step()

print(loss.item())

4.773014545440674


In [98]:
print(decode(m.generate(idx=torch.zeros((1, 1),dtype= torch.long),max_new_tokens=1000)[0].tolist()))


BKSJ?dlIVOv;sA-MCG,SDmu3bWV:XE;n;a!uo,iMjulxJrgZ!wSCKptRj?hiAX&dZVw!oVjHxf flgVkS!puCwWcPiOEj3oXapto:C?a!ufIN3HnSFwJWW:lttXeFurJ$pKHxnOpGMlyjxKBdzr3FEGQLSPvLNL&XHx-Ku
d,!uUzbO'.zC;VcLNukCKwvyeChS&PMpMkI.je
w:&XeZXEJlysBk$kltGGL-lyuQQbo
AkzLIoive?sis,SQW-LBfq,bmhswWkzoIvi
lgOdbUYwb
Sv'QzoHUjX FWk&flgCKHxBObatRoagZaN.lVLUgm!UinwknQ-3:h!u3SlIq-$JvJmRjxa!H&q-WgYdPlukbpxcCKmNQqx;nXUDnMOFNvfy3HOMkmngqpP CJM:CJSP CKRtkzujxq;AtRIq-gkDyhsrYD&ya!.LjRSgRlix
lfxrjQYDCG,bQlnE$s&yu!awWPHocSBivktk!gCD.
o!A-CUQlgGFCbOyZMk$HtUD3SZSPJUtk:ZKzkt$U fxJdT'a!&jjKKCYxqheC-oIfjRhx&y3I!!UzevmZ!BO.Ic3SPHI&o.Sxcp,iyehsrsBzLgXx
GOlbOTbHmPIoTb?YIZvmruxJp'uMprd,lHEQ?IoCF'zDqdZ-mpmnphllgO kzlx?a!mAY:;;;ObCuqSGvreDTtRmNeDnQJZHOYTROTEQLToTrxsrQQqa;G&ZQYxtNxlDnpG ?GFXXS.SUy;zkM?w'V.kzkphsEJ.igmyZY!uBiM?Od?ENEq-bibyx;wA:wa!o?q-oi.i
SylV.l?3nZ?RSV;!zVbTf&AciqfFV&uvbHAXybom;SPzqd,lgXYCOp,vV'nlx?cOdzfIkzSl3QwJmqS.EyYGgvU!U'kGvj$ q,SCP f?YU C&ALzboo' 'T'D.,Ow&lId,s$mJ'ajuR!uJXT'Y.x:wmZgCixzoD :WPyhruOpxOL,s!UN'yZ;
.
wfqwXE$

**Mathematical trick self attention**

In [100]:
#an example of atoy example
torch.manual_seed(1337)
B,T,C = 4,8,2 #batch, time , channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [101]:
#I want x[b,t] = mean{i<t x[b,i]}
xbow = torch.zeros((B,T,C))
for b in range(B):
  for t in range(T):
    xprev = x[b,:t+1] #(T,C)
    xbow[b,t] = torch.mean(xprev,0)